In [26]:
import numpy as np

In [27]:
nodes = 8
edges = 11
MaxDepth = nodes * 2

In [28]:
class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges
        self.graph = {}
    def add_edge_dirless(self, x, y, w):
        self.graph[x] = (y, w)
        self.graph[y] = (x, w)
    def visit_whole_graph(self):
        for v, (u, w) in self.graph.items():
            print("{} to {} costs {}".format(v, u, w))
            

In [29]:
G = Graph(nodes, edges)

G.add_edge_dirless('A', 'B', 3)
G.add_edge_dirless('A', 'H', 4)
G.add_edge_dirless('C', 'B', 4)
G.add_edge_dirless('H', 'B', 5)
G.add_edge_dirless('C', 'G', 3)
G.add_edge_dirless('C', 'D', 8)
G.add_edge_dirless('G', 'H', 2)
G.add_edge_dirless('G', 'D', 8)
G.add_edge_dirless('G', 'F', 4)
G.add_edge_dirless('D', 'E', 2)
G.add_edge_dirless('D', 'F', 3)

In [30]:
G.visit_whole_graph()

A to H costs 4
B to H costs 5
H to G costs 2
C to D costs 8
G to F costs 4
D to F costs 3
F to D costs 3
E to D costs 2
